In [18]:
import pysam
import csv
from pyfaidx import Fasta
import pandas as pd    ll
        Run Cells
        Run Cells and Select Below
        Run Cells and Insert Below
        Run All
        Run All Above
        Run All Below
        Cell Type
        Current Outputs
        All Output
    Kernel
    Widgets
    Help



In [19]:
# A script that takes a list of positions relative to a reference sequence.
# The script then looks at the pileup at each of these positions.
# Input is a sorted bam file
# Output is a dataframe

In [20]:
sam_file_location = 'SRR1918637.sorted.bam'
transcript = '1'
snp_file = 'snp_pos_list.csv'
ref_sequence = '../test_sequences/TPSAB1.fasta'
ref_name = 'NC_000016.10:1240677-1242554'

In [21]:
def get_bases_at_pos(df, transcript, sam_file_location):
    """
    For a given position in the reference what are the bases in the reads which align at \
    that position.
    
    """
    
    samfile = pysam.AlignmentFile(sam_file_location, "rb")
    
    iter = samfile.pileup(contig=transcript)
    
    pos = df['position']
    
    for column in iter:
                
        if column.reference_pos == pos:
            
            base_list = []
              
            for read in column.pileups:

                if read.is_del == 0:

                    base_list.append(read.alignment.query_sequence[read.query_position])

                else:

                    base_list.append('_')
                    
            return ''.join(base_list)

In [22]:
df = pd.read_csv(snp_file)
            
df['pileup'] = df.apply(get_bases_at_pos, axis=1, args=[transcript,sam_file_location])

In [23]:
def get_percent_ref(df, reference, ref_name):
    
    pileup_list = list(df['pileup'])
    
    base_position = df['position']
    
    ref_fasta = Fasta(reference)
    
    reference_base = ref_fasta[ref_name][base_position].seq
    
    ref_count = 0
    
    for base in pileup_list:
        
        if base == reference_base:
            
            ref_count = ref_count +1
            
    return (ref_count / len(pileup_list)) *100

In [24]:
df['pct_ref'] = df.apply(get_percent_ref, axis=1, args=[ref_sequence,ref_name])

TypeError: ("'NoneType' object is not iterable", 'occurred at index 0')

In [ ]:
df.head()